In [2]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt


# Lab 4: Fire and Tree Mortality

# About the data
Wildfires are increasingly frequent and severe due to climate change. Predicting tree mortality following a wildfire is critical for forest management, ecosystem recovery, and carbon sequestration planning. In this lab, we will build a logistic regression model to predict the probability of tree mortality one year after a wildfire

The database we'll be working with today includes observations of individual trees involved in prescribed fires and wildfires occurring over 35 years, from 1981 to 2016. It is drawn from a fire and tree mortality database from the US Forest Service (see data description for the full database here: [link](https://www.nature.com/articles/s41597-020-0522-7#Sec10)).

The target variable we'll use is `yr1status`, which is a binary variable (0=alive, 1=dead).  This tells us if a tree has died one year after a fire event.

The features we'll use are `YrFireName`, `Times_burned`, `Species`, `Genus_species`,
    `DBH_cm`, `HT_m`, `CR_pre`, and `CR_post`.

## Step 1: Check the metadata

Look at the metadata and provide a description on what each variable represents in the Description column below.


| Feature                     | Description                                                                                   |
|-----------------------------|-----------------------------------------------------------------------------------------------| 
| yr1status                   | Binary
| YrFireName                  |                                     
| Times_burned                |                                              
| Species                     |                                                     
| Genus_species               |                                       
| DBH_cm                      | 
| HT_m                        |
| CR_pre                      | 
| CR_post                     |

## Step 2: Fetch  data
Read in the data set and filter to retain only the variables of interest.  Then check for incomplete observations and remove any rows containing NaNs.  How many observations does that leave us with? **Print your answer.**

In [ ]:
# Load the dataset and keep following columns
trees_dat = pd.read_csv('data/FTM_trees.csv', usecols=['yr1status', 'YrFireName', 'Times_burned', 'Species', 'Genus_species', 'DBH_cm', 'HT_m', 'CR_pre', 'CR_post'])

# Drop NAs
trees_dat = trees_dat.dropna()
trees_dat.isna().sum()

YrFireName       0
Species          0
Times_burned     0
Genus_species    0
DBH_cm           0
HT_m             0
CR_pre           0
yr1status        0
CR_post          0
dtype: int64

In [24]:
# View new data frame
trees_dat

,YrFireName,Species,Times_burned,Genus_species,DBH_cm,HT_m,CR_pre,yr1status,CR_post
500,2003 - Griff,ABAM,1,Abies_amabilis,71.374,41.76,0.84,0.0,0.74
501,2003 - Griff,ABAM,1,Abies_amabilis,23.622,12.80,0.60,0.0,0.57
502,2003 - Griff,ABAM,1,Abies_amabilis,46.228,34.75,0.75,0.0,0.59
503,2003 - Griff,ABAM,1,Abies_amabilis,21.082,23.16,0.38,0.0,0.38
504,2003 - Griff,ABAM,1,Abies_amabilis,24.384,26.21,0.42,0.0,0.42
...,...,...,...,...,...,...,...,...,...
169332,2003 - B and B,TSME,1,Tsuga_mertensiana,32.512,15.54,0.90,0.0,0.90
169333,2003 - B and B,TSME,1,Tsuga_mertensiana,24.892,14.63,0.81,0.0,0.81
169334,2003 - B and B,TSME,1,Tsuga_mertensiana,32.258,16.46,0.85,0.0,0.63
169335,2003 - B and B,TSME,1,Tsuga_mertensiana,31.750,18.59,0.87,0.0,0.80


## Step 3: Data Preprocessing
1. We recode categorical predictors to zero-based integer form because most machine learning models, including logistic regression, cannot work directly with categorical data represented as strings or labels. Instead, models require numerical input. Let's do that here. 


In [37]:
trees_dat['YrFireName'] = trees_dat['YrFireName'].astype('category').cat.codes
trees_dat['Species'] = trees_dat['Species'].astype('category').cat.codes
trees_dat['Genus_species'] = trees_dat['Genus_species'].astype('category').cat.codes
trees_dat

,YrFireName,Species,Times_burned,Genus_species,DBH_cm,HT_m,CR_pre,yr1status,CR_post
500,37,0,1,0,71.374,41.76,0.84,0.0,0.74
501,37,0,1,0,23.622,12.80,0.60,0.0,0.57
502,37,0,1,0,46.228,34.75,0.75,0.0,0.59
503,37,0,1,0,21.082,23.16,0.38,0.0,0.38
504,37,0,1,0,24.384,26.21,0.42,0.0,0.42
...,...,...,...,...,...,...,...,...,...
169332,33,31,1,31,32.512,15.54,0.90,0.0,0.90
169333,33,31,1,31,24.892,14.63,0.81,0.0,0.81
169334,33,31,1,31,32.258,16.46,0.85,0.0,0.63
169335,33,31,1,31,31.750,18.59,0.87,0.0,0.80


In [ ]:
trees_dat.dtypes

YrFireName          int8
Species             int8
Times_burned       int64
Genus_species       int8
DBH_cm           float64
HT_m             float64
CR_pre           float64
yr1status        float64
CR_post          float64
dtype: object

In [13]:
# Change CR_pre column to numeric
trees_dat['CR_pre'] = trees_dat['CR_pre'].fillna(0)
trees_dat['CR_pre']

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

2. Then we'll split into training and test data and scale for coefficient interpretability.  Recall that we use the training features to calculate our scaling parameters (mean and standard deviation) and apply the scaling to those training features (`scaler.fit_transform`) and then apply the scaling to the features in the test data as well (`scaler.transform`).


In [ ]:

...

3. How many training/test observations do we have? Print your answer in the cell below. 

In [ ]:
...

# Verify the training and testing set size
print("Training set observations:", )
print("Testing set observations:",)

## Step 4: Train a Logistical Model
Create a classifier using `LogisticRegression()` and fit it on the training data.  Then assess the model's accuracy on the training set by making predictions on the training data.  Calculate and **print** the accuracy of your model on the training set. 

In [ ]:
...
print(f"Training Accuracy: ")

## Step 5: Test Set Predictions and Model Evaluation
Now let's take our trained logistic classifier and make predictions on the test set. Calculate the accuracy and confusion matrix. Then use `sns.heatmap` for improved confusion matrix visualization.

In [ ]:
...

# Print evaluation metrics
print(f"Accuracy: ")


#Plot confusion matrix
...

## Step 6: Logistic Classifier Evaluation
How did your model perform on the unseen data? 
Does your model perform differently on observations of trees that survived vs trees that died?
Is there a class imbalance in this data set?

*Your answer here*

## Step 7: What about a Dummy?
What do you think would happen if we built a model that always predicts the majority class (dead trees)? How would its accuracy compare to your logistic regression model?

*Your answer here*

Let's go ahead and do it: use `DummyClassifier()` with the appropriate value for the 'strategy' parameter to train a majority classifier.  Then calculate this model's accuracy on the training data.

In [ ]:
...

# Print accuracy and confusion matrix results
print(f"Dummy Accuracy: ")
print("\nDummy Confusion Matrix:")


# Step 8: ROCs and AUCs
Our two models have similar accuracy, but is that all there is to this story?  Let's dig a little deeper on the comparison of our logistic and dummy classifiers by examining the associated receiver-operator characteristic (ROC) curves. Calculate the area under the curve (AUC) for both models.

In [ ]:
# Logistic classifier AUC
...
print(f"Logistic AUC: ")

In [ ]:
# Dummy classifier AUC
...
print(f"Dummy AUC: ")

# Step 9: Plot dummy and logistic model ROC curves
Now using the outputs from `roc_curve()`, plot the ROC curves for both models on the same plot.  Make sure to use appropriate labels in the legend.

How do the two models compare on AUC?  What are the implications for evaluating classifiers based on accuracy of their predictions?

*Your answer here*

# Step 10: Final interpretation

Identifying the most important features in a model can guide decision-making. For instance, in our dataset, highly important features might indicate key factors affecting tree survival after a fire. We will calculate the feature importance by examining the coefficients of our logistic regression model.

In [ ]:
...

# Print the sorted feature importance
print(importance_df)

Which are the most important features in our model (reference the metadata to help answer this)? Can you think of any implications for forest management or conservation strategy?

*Your answer here*